In this project I will try to predict which current NBA players will be accepted to the NBA Hall of Fame

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

I have a collection of files with data. I will create dataframes for all of them and then will concentrate all the needed information in one dataframe.

Let me list the features I would rather have in my dataset(everything is in totals for the whole career of a player):

1) pts - number of points scored

2) assists - number of assists made

3) reb - number of rebounds

4) steals - number of steals made

5) blocks - number of blocks made

6) games - number of games played

7) minutes - number of minutes played

8) player - player's name

9) birth_year - year of birth of a player

10) first_seas - first season played

11) last_seas - last season played

12) num_seasons - number of seasons played

13) All NBA 1st team - number of times a player was selected to the All NBA 1st team of the season

14) All NBA 2nd team - number of times a player was selected to the All NBA 2nd team of the season

15) All NBA 3rd team - number of times a player was selected to the All NBA 3rd team of the season

16) All Defense  1st team - number of times a player was selected to the NBA All Defense 1st team of the season

17) All Defense  2nd team - number of times a player was selected to the NBA All Defense 2nd team of the season

18) All Rookie 1st team - number of times a player was selected to the NBA All Rookie 1st team of the season

19) All Rookie 2nd team - number of times a player was selected to the NBA All Rookie 2nd team of the season

20) All ABA 1st team - number of times a player was selected to the All ABA 1st team of the season

21) All ABA 2nd team - number of times a player was selected to the All ABA 2nd team of the season

22) All Star appearances - number of times a player was selected to play in the All Star game

23) MVPs - number of times a player was the Most Valuable Player of a season

24) DPOY - number of times a player was the Defensive Player of the Year 

25) NBA ROY - whether a player was the Rookie of the Year 

26) MIP - whether a player was the Most Improved Player

27) SMOY - number of times a player was the Sixth Man of the Year

28) ABA MVP - number of times a player was the ABA Most Valuable Player

29) ABA ROY - whether a player was the ABA Rookie of the Year

30) Championships - number of championships a player won

31) Finals MVP - number of times a player was the NBA Finals Most Valuable Player

32) Scoring Champ - number of times a player averaged the most points per game in one season

33) NBA Assist Leader - number of times a player averaged the most assists per game in one season

34) NBA Rebounding Leader - number of times a player averaged the most rebounds per game in one season

35) NBA Steal Champ - number of times a player averaged the most steals per game in one season

36) NBA Block Champ - number of times a player averaged the most blocks per game in one season

37) All-Star game MVP - number of times a player was the Most Valuable Player of the All-Star game

38) Conference Finals MVP - number of times a player was the Most Valuable Player of the Conference Finals

39) NBA 75 Team - whether a player is a member of the NBA 75 team

40) ABA All-Time Team - whether a player is a member of the ABA All-Time Team

41) eFG% - effective field goal percentage of a player

42) PER - player efficiency rating

43) hof - whether player is in Hall of Fame







In [ ]:
player_totals=pd.read_csv('../input/nba-aba-baa-stats/Player Totals.csv')
player_season_info=pd.read_csv('../input/nba-aba-baa-stats/Player Season Info.csv')
player_career_info=pd.read_csv('../input/nba-aba-baa-stats/Player Career Info.csv')
player_award_shares=pd.read_csv('../input/nba-aba-baa-stats/Player Award Shares.csv')
all_star=pd.read_csv('../input/nba-aba-baa-stats/All-Star Selections.csv')
advanced=pd.read_csv('../input/nba-aba-baa-stats/Advanced.csv')
end_of_season=pd.read_csv('../input/nba-aba-baa-stats/End of Season Teams.csv')
hof=player_career_info

In [ ]:
hof.columns

In [ ]:
hof

'hof' dataframe has the information about whether player is in Hall of Fame or not, that's what we need

In [ ]:
player_totals

In [ ]:
player_totals.columns

I will extract number of points, assists, rebounds and other valuable totals from this dataset.

In [ ]:
list=hof['player']

points=[]
assists=[]
reb=[]
blocks=[]
steals=[]
games=[]
minutes=[]

for i in list:
    points.append(player_totals[player_totals['player']==i].sum()['pts'])
    assists.append(player_totals[player_totals['player']==i].sum()['ast'])
    reb.append(player_totals[player_totals['player']==i].sum()['trb'])
    blocks.append(player_totals[player_totals['player']==i].sum()['blk'])
    steals.append(player_totals[player_totals['player']==i].sum()['stl'])
    games.append(player_totals[player_totals['player']==i].sum()['g'])
    minutes.append(player_totals[player_totals['player']==i].sum()['mp'])
hof['points']=points
hof['assists']=assists    
hof['reb']=reb
hof['blocks']=blocks
hof['steals']=steals
hof['games']=games
hof['minutes']=minutes


I need to extract info about end of season teams

In [ ]:
allnba1=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-NBA')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==3:
        x=a['number_tm'].value_counts()['1st']
    elif a['number_tm'].value_counts().count()==2:
        if (a['number_tm'].value_counts().index[0]=='1st') | (a['number_tm'].value_counts().index[1]=='1st'):
            x=a['number_tm'].value_counts()['1st']
        else:
            x=0
    else:
        if a['number_tm'].value_counts().index[0]!='1st':
            x=0
        else:
            x=a['number_tm'].value_counts()['1st']
            
    allnba1.append(x)    
hof['All NBA 1st team']=allnba1        

In [ ]:
allnba2=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-NBA')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==3:
        x=a['number_tm'].value_counts()['2nd']
    elif a['number_tm'].value_counts().count()==2:
        if (a['number_tm'].value_counts().index[0]=='2nd') | (a['number_tm'].value_counts().index[1]=='2nd'):
            x=a['number_tm'].value_counts()['2nd']
        else:
            x=0
    else:
        if a['number_tm'].value_counts().index[0]!='2nd':
            x=0
        else:
            x=a['number_tm'].value_counts()['2nd']
            
    allnba2.append(x)    
hof['All NBA 2nd team']=allnba2        

In [ ]:
allnba3=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-NBA')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==3:
        x=a['number_tm'].value_counts()['3rd']
    elif a['number_tm'].value_counts().count()==2:
        if (a['number_tm'].value_counts().index[0]=='3rd') | (a['number_tm'].value_counts().index[1]=='3rd'):
            x=a['number_tm'].value_counts()['3rd']
        else:
            x=0
    else:
        if a['number_tm'].value_counts().index[0]!='3rd':
            x=0
        else:
            x=a['number_tm'].value_counts()['3rd']
            
    allnba3.append(x)    
hof['All NBA 3rd team']=allnba3        

In [ ]:
allnbadef=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-Defense')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['1st']
    else:
        if a['number_tm'].value_counts().index[0]!='1st':
            x=0
        else:
            x=a['number_tm'].value_counts()['1st']
            
    allnbadef.append(x)    
hof['All Defense 1st team']=allnbadef        

In [ ]:
allnbadef2=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-Defense')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['2nd']
    else:
        if a['number_tm'].value_counts().index[0]!='2nd':
            x=0
        else:
            x=a['number_tm'].value_counts()['2nd']
            
    allnbadef2.append(x)    
hof['All Defense 2nd team']=allnbadef2        

In [ ]:
allrooks=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-Rookie')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['1st']
    else:
        if a['number_tm'].value_counts().index[0]!='1st':
            x=0
        else:
            x=a['number_tm'].value_counts()['1st']
            
    allrooks.append(x)    
hof['All Rookie 1st team']=allrooks        

In [ ]:
allrooks2=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-Rookie')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['2nd']
    else:
        if a['number_tm'].value_counts().index[0]!='2nd':
            x=0
        else:
            x=a['number_tm'].value_counts()['2nd']
            
    allrooks2.append(x)    
hof['All Rookie 2nd team']=allrooks2        

In [ ]:
allaba=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-ABA')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['1st']
    else:
        if a['number_tm'].value_counts().index[0]!='1st':
            x=0
        else:
            x=a['number_tm'].value_counts()['1st']
            
    allaba.append(x)    
hof['All ABA 1st team']=allaba        

In [ ]:
allaba2=[]
for i in list:
    a=end_of_season[(end_of_season['type']=='All-ABA')&(end_of_season['player']==i)]
    if a.empty:
        x=0     
    elif a['number_tm'].value_counts().count()==2:
        x=a['number_tm'].value_counts()['2nd']
    else:
        if a['number_tm'].value_counts().index[0]!='2nd':
            x=0
        else:
            x=a['number_tm'].value_counts()['2nd']
            
    allaba2.append(x)    
hof['All ABA 2nd team']=allaba2        

In [ ]:
allstar=[]
for i in list:
    a=all_star[all_star['player']==i]
    if a.empty:
        x=0     
    else:
        x=a['replaced'].count()
            
    allstar.append(x)    
hof['All Star appearances']=allstar        

In [ ]:
mvp=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='nba mvp')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    mvp.append(x)    
hof['MVPs']=mvp        

In [ ]:
dpoy=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='dpoy')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    dpoy.append(x)    
hof['DPOY']=dpoy        

In [ ]:
nba_roy=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='nba roy')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    nba_roy.append(x)    
hof['NBA ROY']=nba_roy        

In [ ]:
mip=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='mip')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    mip.append(x)    
hof['MIP']=mip        

In [ ]:
smoy=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='smoy')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    smoy.append(x)    
hof['SMOY']=smoy        

In [ ]:
aba_mvp=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='aba mvp')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    aba_mvp.append(x)    
hof['ABA MVP']=aba_mvp        

In [ ]:
aba_rook=[]
for i in list:
    a=player_award_shares[(player_award_shares['award']=='aba roy')&(player_award_shares['player']==i)]
    if a.empty:
        x=0     
    else:
        x=sum(a['winner'])
            
    aba_rook.append(x)    
hof['ABA ROY']=aba_rook        

Now I need to extract data from 'NBA_players_clean.csv' file, I need information about number of championships, scoring champs and etc.

In [ ]:
clean=pd.read_csv('../input/all-nba-aba-players-bio-stats-accolades/NBA_players_clean.csv')
clean.columns

In [ ]:
clean['Player']

Some players have asterisks after their name.

In [ ]:
def cut_string(str):
    if '*' in str:
        return str.split('*')[0]
    else:
        return str
clean['player']=clean['Player'].apply(lambda x:cut_string(x))
    


In [ ]:
clean['player']

In [ ]:
list=hof['player']
list5=clean['player']

In [ ]:
rings_list=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['Championships']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    rings_list.append(x)    
            
hof['Championships']=rings_list        
        

In [ ]:
fmvp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['Finals MVP']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    fmvp.append(x)    
            
hof['Finals MVP']=fmvp        
        

In [ ]:
scorer=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['Scoring Champ']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    scorer.append(x)    
hof['Scoring Champ']=scorer            
        
        

In [ ]:
asstchmp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['AST Champ']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    asstchmp.append(x)    
hof['NBA Assist Leader']=asstchmp            
        
        

In [ ]:
rebchmp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['TRB Champ']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    rebchmp.append(x)    
hof['NBA Rebounding Leader']=rebchmp            
        
        

In [ ]:
stlchmp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['STL Champ']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    stlchmp.append(x)    
            
hof['NBA Steal Champ']=stlchmp        
        

In [ ]:
blkchmp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['BLK Champ']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    blkchmp.append(x)    
hof['NBA Block Champ']=blkchmp            
        
        

In [ ]:
asmvp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['AS MVP']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    asmvp.append(x)    
hof['All-Star game MVP']=asmvp            
        
        

In [ ]:
cfmvp=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['CF MVP']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    cfmvp.append(x)    
            
hof['Conference Finals MVP']=cfmvp     
        

In [ ]:
nba75=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['NBA 75 Team']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    nba75.append(x)    
            
hof['NBA 75 Team']=nba75        
        

In [ ]:
abaaltime=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['ABA All-Time Team']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    abaaltime.append(x)    
hof['ABA All-Time Team']=abaaltime            
        
        

In [ ]:
efg=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['eFG%']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    efg.append(x)    
hof['eFG%']=efg            
        
        

In [ ]:
per=[]
for i in list:
    for j in list5:
        
        if j==i:
            a=clean[clean['player']==j]['PER']
            if a.empty:
                x=0
            else:
                x=a.iloc[0]
        
    per.append(x)    
hof['PER']=per            
        
        

Let's see what we have now.

In [ ]:
hof.head()

In [ ]:
hof.columns

Let's change True and False in our target column to 1 and 0.

In [ ]:
hof['hof'] = hof['hof'].astype(int)

I checked important columns on having NaN values and found two such columns.

In [ ]:
hof['SMOY'].isnull().sum()

In [ ]:
hof['MIP'].isnull().sum()

In [ ]:
hof['MIP'] = hof['MIP'].fillna(0)
hof['SMOY'] = hof['SMOY'].fillna(0)

One more thing, for some reason Patrick Ewing has two columns in the dataset. I will drop the one that states that his first season was in 2011 which is not true.

In [ ]:
hof[hof['player']=='Patrick Ewing']['first_seas'].size

In [ ]:
hof.drop(3966,inplace=True)

Now let's do some analysis

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='num_seasons',data=hof,palette='coolwarm')

Looks like almost all Hall of Famers have at least 10 seasons played

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='first_seas',data=hof,palette='coolwarm')

Most of all Hall of Famers started their careers before 1990 which is understandable because acceptance to the Hall of Fame happens not right after the end of the career of a player but after some years. That's why a lot of current players are not in Hall of Fame yet.

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='points',data=hof,palette='coolwarm')

Looks like points and getting in HOF might have a good correlation. Assists, steals, rebounds, blocks have a slightly worse correlation with target. 

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='assists',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='reb',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='steals',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='blocks',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='games',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='minutes',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='All NBA 1st team',data=hof,palette='coolwarm')

Most of Hall of Famers have been in the 1st or 2nd ALL NBA team in the end of a season.

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='All NBA 2nd team',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='All NBA 3rd team',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='All Defense 1st team',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='All Star appearances',data=hof,palette='coolwarm')

Seems like most of Hall of Famers have been All Stars at least 4 times.

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='MVPs',data=hof,palette='coolwarm')

In [ ]:
fig, ax = plt.subplots(figsize=(13,8))
sns.boxplot(x='hof',y='Championships',data=hof,palette='coolwarm')

Championships play some role too.

In [ ]:
px.scatter(
        hof, x="points", y="eFG%", color='hof', 
        hover_data=['player'])

In [ ]:
px.scatter(
        hof, x="PER", y="games", color='hof', 
        hover_data=['player'])

In [ ]:
px.scatter(
        hof[hof['games']>300], x="points", y="eFG%", color='hof', 
        hover_data=['player'])

In [ ]:
px.scatter(
        hof[hof['games']>200], x="PER", y="minutes", color='hof', 
        hover_data=['player'])

In [ ]:
px.scatter(
        hof, x="points", y="assists", color='hof', 
        hover_data=['player'])

LeBron's stats stands out(upper-right area).

In [ ]:
px.scatter(
        hof, x="points", y="reb", color='hof', 
        hover_data=['player'])

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(hof[['num_seasons','first_seas','points','assists','reb','blocks','minutes','All NBA 1st team','All NBA 2nd team',
                'All Defense 1st team','All Star appearances','MVPs','Championships','Scoring Champ',
                 'NBA 75 Team','steals','DPOY','eFG%','PER','hof']].corr(),annot=True)

Now let's do machine learning and predictions. I am going to separate the dataset, players that started their careers before 2000s will be in the training data slice. That way we will see which current players our model will have as Hall of Famers based on their career achievements.

In [ ]:
young=hof[hof['first_seas']>2000]
old=hof[hof['first_seas']<2000]
X_train=old.drop(['birth_year','player_id','first_seas','last_seas','hof','player'],axis=1)
y_train=old['hof']
X_test=young.drop(['birth_year','player_id','first_seas','last_seas','hof','player'],axis=1)
y_test=young['hof']

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred_knn = knn.predict(X_test)
knn_prob=knn.predict_proba(X_test)[:,0-1]

logmodel=LogisticRegression(max_iter=2000)
logmodel.fit(X_train,y_train)
pred_log=logmodel.predict(X_test)
log_prob=logmodel.predict_proba(X_test)[:,0-1]

rfc=RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
pred_rfc=rfc.predict(X_test)
rfc_prob=rfc.predict_proba(X_test)[:,0-1]


print('Logistic Regression')
print(confusion_matrix(y_test,pred_log))
print(classification_report(y_test,pred_log))

print('Random Forest')
print(confusion_matrix(y_test,pred_rfc))
print(classification_report(y_test,pred_rfc))


print('K Nearest Neighbor')
print(confusion_matrix(y_test,pred_knn))
print(classification_report(y_test,pred_knn))




Let's see which players were predicted to be Hall of Famers

In [ ]:
res=pd.concat([X_test,y_test],axis=1)

res['log_pred']=pred_log
res['log_prob']=log_prob

res['rfc_pred']=pred_rfc
res['rfc_prob']=rfc_prob

res['knn_pred']=pred_knn
res['knn_prob']=knn_prob

res_final=pd.concat([res,hof['player']],axis=1)

In [ ]:
res_final[(res_final['hof']==0)&((res_final['log_pred']==1)|(res_final['knn_pred']==1)|(res_final['rfc_pred']==1))].head(60)

It's clear that Logistic Regression model and Random Forest model made more realistic predictions than KNN model which classified players like Enes Freedom or Eric Bledsoe as possible Hall of Famers. Also KNN model failed to classify any player of 3 players that already got into Hall of Fame.

Let's see which players were left out. I will sort those who have more than 3 All Star appearances since we saw that this is the most correlated feature to the target column.

In [ ]:
res_final[(res_final['hof']==0)&((res_final['log_pred']==0)|(res_final['rfc_pred']==0))&(res_final['All Star appearances']>3)]

Jimmy Butler stands out since not any model predicted him get into the Hall of Fame. Perhaps the reason is that Jimmy usually shines more in Playoffs so his regular season achievements don't help him here. 
Let's see the one player that was no classified as HOFer although he actually is.

In [ ]:
res_final[(res_final['hof']==1)&((res_final['log_pred']==0)|(res_final['rfc_pred']==0))]

So it is Manu Ginobili who's acceptance ceremony was three weeks ago. He is actually very close by his probability numbers to be counted as HOFer by the models. Let's see his achievements:

In [ ]:
hof.iloc[3400]

So, only two All Star appearances which might be the reason why models classified him as not a HOFer. He has 4 championships that is probably what gave him the probability close to 0.5.